In [ ]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Get GitHub repository

from urllib.request import urlopen
from zipfile import ZipFile

repo_url = 'https://github.com/ma-lab-cgidr/zebrafish-quantification/archive/refs/heads/master.zip'
with open('/tmp/repo.zip', 'wb') as zip_file:
  response = urlopen(repo_url)
  zip_file.write(response.read())

with ZipFile('/tmp/repo.zip') as zip_file:
  zip_file.extractall(path = '/tmp/')

In [ ]:
!pip install -r /tmp/zebrafish-quantification-master/requirements.txt

Ok, now restart the runtime (Ctrl+M; or, in the top left menu, Runtime > Restart runtime). This is so that packages installed in the previous step are available for use within this python runtime.

In [ ]:
import glob
import os
import sys

sys.path.append('/tmp/zebrafish-quantification-master')

import pipeline

Configure the following settings to match the experiment being analyzed.

In [ ]:
# Common Parameters: Change these for every experiment

CHECKERBOARD = True                                     # set to false if doing simple dose-response curve(s)
CONVERSIONS = {                                         # fill in drug dose conversions used in this experiment
    'NEO50': 'NEO 4.8μM',
    'NEO99': 'NEO 24.2μM'
}
DATE = '2022-07-13'                                     # enter the date of the experiment, so files created have meaningful names
FOLDER = '/content/drive/MyDrive/2022-07-13_NeoScreen'  # fill in which directory contains the images for this experiment

# Uncommon Parameters: We tend to leave these the same, but modify if needed

PLATE_CONTROL = 'Control 0μM'
PLATE_POSITIVE_CONTROl = 'NEO99'
PLATEFILE = FOLDER + '/plate-template-all.csv'

# Generated Parameters: You probably don't need to change these, but you can

chartfile = f'{FOLDER}/chart_{DATE}.png'
imagefiles = glob.glob(f'{FOLDER}/*/*_CH1.tif')

In [ ]:
# Configure output to go in a helpful location

config_file = '/tmp/zebrafish-quantification-master/config-ext.ini'

for dir in ('.cache', 'dose_response', 'imageops'):
  os.makedirs(f'{FOLDER}/log/{dir}', exist_ok=True)

with open(config_file, 'w') as file:
    file.write('[Main]\n')
    file.write(f'log_dir = {FOLDER}/log\n')

# Look the other way... this is super janky. But it's needed atm for files to
# end up in the proper location
sys.argv[0] = '/tmp/zebrafish-quantification-master/pipeline.py'

In [ ]:
# Go ahead and run the pipeline!

pipeline.main(imagefiles, chartfile=chartfile, checkerboard=CHECKERBOARD,
              conversions=CONVERSIONS, platefile=PLATEFILE,
              plate_control=[PLATE_CONTROL], plate_info=DATE,
              plate_positive_control=[PLATE_POSITIVE_CONTROl])